In [ ]:
!pip install colabcode
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install python-multipart
!pip install aiofiles

In [ ]:
!git clone https://github.com/NathanUA/U-2-Net.git
!pip install gdown
!gdown https://drive.google.com/u/1/uc?id=1IG3HdpcRiDoWNookbncQjeaPN28t90yW
!mkdir ./saved_models
!mkdir ./saved_models/u2net_portrait
!mkdir ./test_data
!mkdir ./test_data/test_portrait_images
!mkdir ./test_data/test_portrait_images/your_portrait_im
!mkdir ./test_data/test_portrait_images/portrait_im
!cp u2net_portrait.pth ./saved_models/u2net_portrait

In [2]:
from colabcode import ColabCode
server = ColabCode(port=10000, code=False)
from fastapi import FastAPI, File, UploadFile, Request
import pickle
import shutil
from io import BytesIO
from typing import List
from fastapi.responses import StreamingResponse
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import HTMLResponse
from fastapi.responses import FileResponse
from pydantic import BaseModel
from PIL import Image
import os
import aiofiles
import base64
import json
app = FastAPI()

def decodeImg(encodedImg):
    pil_image=Image.open(BytesIO(encodedImg))
    return pil_image
@app.post("/files/")
async def image(file:bytes=File(...)):
    filename="./test_data/test_portrait_images/portrait_im/destination.jpeg"
    image=decodeImg(file)

    !rm -rf ./test_data/test_portrait_images/portrait_im
    !rm -rf ./test_data/test_portrait_images/portrait_results
    !mkdir ./test_data/test_portrait_images/portrait_im
    
    image=image.save(filename)

    !python /content/U-2-Net/u2net_portrait_test.py

    arr=os.listdir('./test_data/test_portrait_images/portrait_results')
    # print(arr)
    with open("./test_data/test_portrait_images/portrait_results/destination.png", "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
        print(encoded_string)
        return {"image":encoded_string}
    # return {FileResponse("./test_data/test_portrait_images/portrait_results/destination.png",media_type="image/jpeg")}
    
@app.get("/")
async def main():
    content = """
<body>
<form action="/files/" enctype="multipart/form-data" method="post">
<input name="files" type="image" >
<input type="submit">
</form>
<form action="/uploadfiles/" enctype="multipart/form-data" method="post">
<input name="files" type="file" multiple>
<input type="submit">
</form>
</body>
    """
    return HTMLResponse(content=content)



In [ ]:
server.run_app(app)

INFO:     Started server process [1216]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "http://02934394df4c.ngrok.io" -> "http://localhost:10000"
INFO:     2409:4043:2ba7:dfde:2098:77d:8b41:ff29:0 - "GET / HTTP/1.1" 200 OK
INFO:     2409:4043:2ba7:dfde:2098:77d:8b41:ff29:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2409:4043:2ba7:dfde:2098:77d:8b41:ff29:0 - "POST /files HTTP/1.1" 307 Temporary Redirect
Number of images:  1
...load U2NET---173.6 MB
inferencing: destination.jpeg
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2952: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation